In [35]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


In [36]:
# data_dir="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small"
# file_path="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small/000/000002.mp3"
train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_combined.csv'
# train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_wav2vec.csv'


In [37]:
data_df=pd.read_csv(train_path, index_col=0)
# data_df=pd.read_csv(train_path)

In [38]:
display(data_df.head())

,0,1,2,3,4,5,6,7,8,9,...,1176,1177,1178,1179,1180,1181,1182,1183,1184,genre
track_id,,,,,,,,,,,,,,,,,,,,,
2,-122.713936,117.760094,-42.334183,38.061005,-23.012323,23.920383,-11.944865,13.687291,-7.294582,4.041143,...,0.000032,0.000033,0.000039,0.000048,0.000078,0.000110,0.000089,0.000050,0.000029,Hip-Hop
5,-162.232422,131.435989,-18.426781,51.631371,-15.444844,23.860094,-7.867664,10.686234,-0.314591,12.864646,...,0.000039,0.000041,0.000060,0.000092,0.000102,0.000146,0.000153,0.000072,0.000038,Hip-Hop
10,-75.631538,154.537613,-63.849487,22.506426,2.425359,10.459981,-13.007332,9.818939,-7.640783,3.136315,...,0.000056,0.000066,0.000069,0.000083,0.000095,0.000103,0.000103,0.000072,0.000058,Pop
140,-322.997864,133.684906,15.497437,54.755615,8.718211,37.022747,-2.116411,15.265619,6.175477,-2.599205,...,0.000041,0.000035,0.000031,0.000038,0.000050,0.000052,0.000034,0.000024,0.000022,Folk
141,-253.314560,157.769379,-6.081675,26.920168,6.813843,15.842272,-12.930041,10.373278,-0.609945,-8.300909,...,0.000029,0.000025,0.000026,0.000048,0.000095,0.000081,0.000036,0.000029,0.000023,Folk


In [39]:

X = StandardScaler().fit_transform(data_df.drop('genre', axis=1))
y = pd.factorize(data_df['genre'])[0]  # Convert genre to numerical labels


In [40]:
display(y)

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [42]:
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

X_train_tensor_rnn = X_train_tensor.unsqueeze(1).expand(-1, 10, -1)  # Shape becomes [batch_size, 1, num_features]
X_test_tensor_rnn = X_test_tensor.unsqueeze(1).expand(-1, 10, -1)  # Shape becomes [batch_size, 1, num_features]
train_dataset_rnn = TensorDataset(X_train_tensor_rnn, y_train_tensor)
test_dataset_rnn = TensorDataset(X_test_tensor_rnn, y_test_tensor)
train_loader_rnn = DataLoader(train_dataset_rnn, batch_size=256, shuffle=True)
test_loader_rnn = DataLoader(test_dataset_rnn, batch_size=256, shuffle=False)


In [43]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        
    def forward(self, x):
        return self.layers(x)


In [44]:
class CNN1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN1D, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * (input_dim // 4), 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc_layers(x)


In [45]:
class RNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, num_layers=1):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])  # Take the last time step
        return x

class LSTM(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, num_layers=1):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x, (hn, cn) = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take the last time step
        return x


In [46]:
class EnsembleModel(nn.Module):
    def __init__(self, mlp, cnn, rnn, lstm, output_dim):
        super(EnsembleModel, self).__init__()
        self.mlp = mlp
        self.cnn = cnn
        self.rnn = rnn
        self.lstm = lstm
        self.fc = nn.Linear(output_dim * 4, output_dim)  # Assuming each model outputs the class probabilities
    
    def forward(self, x):
        out_mlp = self.mlp(x)
        out_cnn = self.cnn(x)
        out_rnn = self.rnn(x.unsqueeze(1))  # Assume RNN/LSTM expects sequence data
        out_lstm = self.lstm(x.unsqueeze(1))
        
        # Concatenate along the feature dimension
        combined = torch.cat((out_mlp, out_cnn, out_rnn, out_lstm), dim=1)
        output = self.fc(combined)
        return output


In [47]:
def train_model(model, train_loader, test_loader, device, epochs=100):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    acc=0

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        total = correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        acc= max(acc,100 * correct / total)

        print(f'Epoch {epoch+1}: Accuracy: {100 * correct / total}%')
    return(acc)

In [48]:
input_size = X_train.shape[1]
num_classes = len(np.unique(y))

In [49]:
model_mlp=MLP(input_size,num_classes).to(device)
train_model(model_mlp, train_loader, test_loader, device)
# 53.87693846923462 entropy loss

Epoch 1: Accuracy: 24.262131065532767%
Epoch 2: Accuracy: 39.1695847923962%
Epoch 3: Accuracy: 43.82191095547774%
Epoch 4: Accuracy: 43.621810905452726%
Epoch 5: Accuracy: 46.673336668334166%
Epoch 6: Accuracy: 47.9239619809905%
Epoch 7: Accuracy: 48.924462231115555%
Epoch 8: Accuracy: 49.22461230615308%
Epoch 9: Accuracy: 49.02451225612806%
Epoch 10: Accuracy: 52.02601300650325%
Epoch 11: Accuracy: 49.67483741870935%
Epoch 12: Accuracy: 49.92496248124062%
Epoch 13: Accuracy: 53.076538269134566%
Epoch 14: Accuracy: 50.125062531265634%
Epoch 15: Accuracy: 51.1255627813907%
Epoch 16: Accuracy: 49.77488744372186%
Epoch 17: Accuracy: 50.82541270635318%
Epoch 18: Accuracy: 50.22511255627814%
Epoch 19: Accuracy: 50.42521260630315%
Epoch 20: Accuracy: 51.42571285642821%
Epoch 21: Accuracy: 53.076538269134566%
Epoch 22: Accuracy: 49.82491245622811%
Epoch 23: Accuracy: 50.97548774387194%
Epoch 24: Accuracy: 51.62581290645323%
Epoch 25: Accuracy: 51.37568784392196%
Epoch 26: Accuracy: 51.7758879

53.87693846923462

In [50]:
model_cnn=CNN1D(input_size,num_classes).to(device)
train_model(model_cnn, train_loader, test_loader, device) 
# 51.52576288144072 entropy loss

c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester6\460j\460jFinalProject\.venv\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch 1: Accuracy: 29.31465732866433%
Epoch 2: Accuracy: 33.616808404202104%
Epoch 3: Accuracy: 35.1175587793897%
Epoch 4: Accuracy: 39.41970985492746%
Epoch 5: Accuracy: 38.71935967983992%
Epoch 6: Accuracy: 39.619809904952476%
Epoch 7: Accuracy: 41.620810405202604%
Epoch 8: Accuracy: 40.420210105052526%
Epoch 9: Accuracy: 42.62131065532766%
Epoch 10: Accuracy: 43.52176088044022%
Epoch 11: Accuracy: 42.87143571785893%
Epoch 12: Accuracy: 44.022011005502755%
Epoch 13: Accuracy: 45.37268634317159%
Epoch 14: Accuracy: 45.37268634317159%
Epoch 15: Accuracy: 45.32266133066533%
Epoch 16: Accuracy: 47.473736868434216%
Epoch 17: Accuracy: 46.07303651825913%
Epoch 18: Accuracy: 46.3231615807904%
Epoch 19: Accuracy: 46.523261630815405%
Epoch 20: Accuracy: 46.37318659329665%
Epoch 21: Accuracy: 47.42371185592796%
Epoch 22: Accuracy: 47.473736868434216%
Epoch 23: Accuracy: 47.073536768384194%
Epoch 24: Accuracy: 46.923461730865434%
Epoch 25: Accuracy: 47.873936968484244%
Epoch 26: Accuracy: 48.67

51.52576288144072

In [51]:
model_rnn=RNN(input_size,num_classes).to(device)
train_model(model_rnn, train_loader_rnn, test_loader_rnn, device) 
#50.17508754377189 entropy loss

Epoch 1: Accuracy: 28.96448224112056%
Epoch 2: Accuracy: 32.36618309154577%
Epoch 3: Accuracy: 35.41770885442721%
Epoch 4: Accuracy: 36.7183591795898%
Epoch 5: Accuracy: 39.01950975487744%
Epoch 6: Accuracy: 40.020010005002504%
Epoch 7: Accuracy: 40.670335167583794%
Epoch 8: Accuracy: 41.32066033016508%
Epoch 9: Accuracy: 42.57128564282141%
Epoch 10: Accuracy: 43.82191095547774%
Epoch 11: Accuracy: 44.072036018009%
Epoch 12: Accuracy: 44.57228614307154%
Epoch 13: Accuracy: 44.47223611805903%
Epoch 14: Accuracy: 45.02251125562781%
Epoch 15: Accuracy: 45.472736368184094%
Epoch 16: Accuracy: 45.472736368184094%
Epoch 17: Accuracy: 46.3231615807904%
Epoch 18: Accuracy: 46.87343671835918%
Epoch 19: Accuracy: 46.57328664332166%
Epoch 20: Accuracy: 47.22361180590295%
Epoch 21: Accuracy: 47.77388694347174%
Epoch 22: Accuracy: 47.67383691845923%
Epoch 23: Accuracy: 47.57378689344672%
Epoch 24: Accuracy: 47.723861930965484%
Epoch 25: Accuracy: 47.9239619809905%
Epoch 26: Accuracy: 48.37418709354

50.17508754377189

In [52]:
model_lstm=LSTM(input_size,num_classes).to(device)
train_model(model_lstm, train_loader_rnn, test_loader_rnn, device) 
#49.67483741870935 entropy loss

Epoch 1: Accuracy: 26.56328164082041%
Epoch 2: Accuracy: 29.864932466233117%
Epoch 3: Accuracy: 32.566283141570786%
Epoch 4: Accuracy: 34.71735867933967%
Epoch 5: Accuracy: 35.91795897948975%
Epoch 6: Accuracy: 36.918459229614804%
Epoch 7: Accuracy: 38.3191595797899%
Epoch 8: Accuracy: 38.76938469234617%
Epoch 9: Accuracy: 39.66983491745873%
Epoch 10: Accuracy: 40.57028514257129%
Epoch 11: Accuracy: 41.77088544272136%
Epoch 12: Accuracy: 42.22111055527764%
Epoch 13: Accuracy: 42.271135567783894%
Epoch 14: Accuracy: 42.271135567783894%
Epoch 15: Accuracy: 42.72136068034017%
Epoch 16: Accuracy: 43.42171085542771%
Epoch 17: Accuracy: 43.67183591795898%
Epoch 18: Accuracy: 44.17208604302151%
Epoch 19: Accuracy: 44.17208604302151%
Epoch 20: Accuracy: 44.422211105552776%
Epoch 21: Accuracy: 45.12256128064032%
Epoch 22: Accuracy: 45.52276138069035%
Epoch 23: Accuracy: 45.42271135567784%
Epoch 24: Accuracy: 45.92296148074037%
Epoch 25: Accuracy: 46.17308654327164%
Epoch 26: Accuracy: 46.873436

49.67483741870935

In [53]:
model_cnn_2=CNN1D(input_size,num_classes).to(device)
model_mlp_2=MLP(input_size,num_classes).to(device)
model_rnn_2=RNN(input_size,num_classes).to(device)
model_lstm_2=LSTM(input_size,num_classes).to(device)

model_ens=EnsembleModel(mlp=model_mlp_2,cnn=model_cnn_2,rnn=model_rnn_2,lstm=model_lstm_2,output_dim=num_classes).to(device)
train_model(model_ens, train_loader, test_loader, device)
# 53.62681340670335 accuracy entropy loss

Epoch 1: Accuracy: 29.514757378689346%
Epoch 2: Accuracy: 35.41770885442721%
Epoch 3: Accuracy: 39.01950975487744%
Epoch 4: Accuracy: 40.72036018009005%
Epoch 5: Accuracy: 43.82191095547774%
Epoch 6: Accuracy: 44.77238619309655%
Epoch 7: Accuracy: 46.023011505752876%
Epoch 8: Accuracy: 47.323661830915455%
Epoch 9: Accuracy: 47.42371185592796%
Epoch 10: Accuracy: 48.37418709354677%
Epoch 11: Accuracy: 50.02501250625313%
Epoch 12: Accuracy: 46.97348674337169%
Epoch 13: Accuracy: 51.82591295647824%
Epoch 14: Accuracy: 49.77488744372186%
Epoch 15: Accuracy: 51.92596298149075%
Epoch 16: Accuracy: 50.22511255627814%
Epoch 17: Accuracy: 50.675337668834416%
Epoch 18: Accuracy: 50.17508754377189%
Epoch 19: Accuracy: 50.675337668834416%
Epoch 20: Accuracy: 52.32616308154077%
Epoch 21: Accuracy: 51.475737868934466%
Epoch 22: Accuracy: 51.725862931465734%
Epoch 23: Accuracy: 51.475737868934466%
Epoch 24: Accuracy: 50.62531265632816%
Epoch 25: Accuracy: 50.77538769384692%
Epoch 26: Accuracy: 51.275

53.62681340670335